In [ ]:
%aiida

In [ ]:
from ipywidgets import interact
from aiida.orm import Code


In [ ]:
code = None
def select_code(codename):
    global code
    code = Code.get_from_string(codename)

code_choice = interact(select_code, 
                       codename=Code.list_for_plugin('siesta.siesta'))

In [ ]:
from aiida.orm import DataFactory, CalculationFactory
SiestaCalculation = CalculationFactory('siesta.siesta')
PsfData = DataFactory('siesta.psf')
StructureData = DataFactory('structure')
ParameterData = DataFactory('parameter')
KpointsData = DataFactory('array.kpoints')


In [ ]:
from aiida.orm.data.base import Float, Str
import os
import numpy as np


In [ ]:
from aiida.work.process_registry import ProcessRegistry
from aiida.work.run import run
from aiida.work.workfunction import workfunction as wf


In [ ]:
@wf
def create_structure(alat):
    """ Create Si-diamond structure with `alat` lattice parameter (distorted). """
    cell = np.array([[0.5, 0.5, 0.0,],
                     [0.0, 0.5, 0.5,],
                     [0.5, 0.0, 0.5,],]) * alat
    structure = StructureData(cell=cell)
    structure.append_atom(
        position=(0.000 * alat, 0.000 * alat, 0.000 * alat),
        symbols=['Si'])
    structure.append_atom(
        position=(0.250 * alat, 0.250 * alat, 0.250 * alat),
        symbols=['Si'])

    return structure


In [ ]:
def geninputs(structure):
    inputs = SiestaCalculation.process().get_inputs_template()

    # Attach structure
    inputs.structure = structure

    # Attach code
    global code
    inputs.code = code
    inputs._options.resources = {
        "num_machines": 1,
        "num_mpiprocs_per_machine": 1,
    }
    inputs._options.max_wallclock_seconds = 30 * 60

    kpoints = KpointsData()
    kpoints_mesh = 4
    kpoints.set_kpoints_mesh([kpoints_mesh, kpoints_mesh, kpoints_mesh])
    inputs.kpoints = kpoints

    # Calculation parameters
    parameters_dict = {
        'xc-functional': 'LDA',
        'xc-authors': 'CA',
        'max-scfiterations': 50,
        'dm-numberpulay': 4,
        'dm-mixingweight': 0.3,
        'dm-tolerance': 1.e-3,
        'Solution-method': 'diagon',
        'electronic-temperature': '25 meV',
        'md-typeofrun': 'cg',
        'md-numcgsteps': 3,
        'md-maxcgdispl': '0.1 Ang',
        'md-maxforcetol': '0.04 eV/Ang',
        'xml-write': True,
    }
    inputs.parameters = ParameterData(dict=parameters_dict)

    # Pseudopotentials
    raw_pseudos = [("Si.psf", 'Si')]
    pseudo_dict = {}
    for fname, kind in raw_pseudos:
        absname = os.path.realpath(
            os.path.join(os.getcwd(), 'pseudos', fname))
        pseudo, created = PsfData.get_or_create(absname, use_first=True)

    if created:
        print "Created the pseudo for {}".format(kind)
    else:
        print "Using the pseudo for {} from DB: {}".format(kind, pseudo.pk)
    # Attach pseudo node to the calculation
    pseudo_dict[kind] = pseudo

    inputs.pseudo = pseudo_dict

    # Basis set
    inputs.basis = ParameterData(dict={
        'pao-energy-shift': '300 meV',
        '%block pao-basis-sizes': 'Si DZP',
    })

    return inputs

In [ ]:
def get_info(calc_results, structure):
    return (structure.get_cell_volume(),
            calc_results['output_parameters'].dict.FreeE)

In [ ]:
alat_range = np.arange(5.35, 5.49, 0.02)
alat_range

In [ ]:
@wf
def run_wf():
    print "Workfunction node identifiers: {}".format(ProcessRegistry().current_calc_node)
    # wcalc_uuid = ProcessRegistry().current_calc_node.uuid
    #print "Workfunction node: {}".format(wcalc_uuid)
    JobCalc = SiestaCalculation.process()
    calcs = {}
    for alat in alat_range:
        structure = create_structure(Float(alat))
        inputs = geninputs(structure)
        print "Running a scf for Si with lattice constant {}".format(alat)
        result = run(JobCalc, **inputs)
        calcs[str(alat)] = get_info(result, structure)

    eos = []
    for alat in alat_range:
        eos.append(calcs[str(alat)])

    retdict = {'result': ParameterData(dict={'eos_data': eos})}

    return retdict

In [ ]:
! verdi daemon restart

In [ ]:
res = run_wf()
eos_data = res['result'].get_attr('eos_data')

eos_data

In [ ]:
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt


def eos_murnaghan(vol, E0, B0, BP, V0):
    """ Murnaghan equation of state (energy as a function of volume).
    From PRB 28,5480 (1983). """
   
    return E0 + B0*vol/BP*(((V0/vol)**BP)/(BP-1)+1) - V0*B0/(BP-1)


def fit_murnaghan(volume, energy):
    """ Function that fits the results to a Murnaghan EOS. """

    # fit a parabola for initial parameter guess
    p_coefs = np.polyfit(volume, energy, 2)
    # minimum of the parabola dE/dV = 0 ( p_coefs = [c,b,a] )
    p_min = - p_coefs[1]/(2.*p_coefs[0])
    # warn if min volume not in result range
    if (p_min < volume.min() or p_min > volume.max()):
        print "Warning: minimum volume not in range of results"
    # goundstate energy estimation form parabola minimum
    E0 = np.polyval(p_coefs, p_min)
    # bulk modulus estimation
    B0 = 2.*p_coefs[2]*p_min

    # initial parameters (BP is usually small)
    init_par = [E0, B0, 4, p_min]
    best_par, cov_matrix = curve_fit(eos_murnaghan, volume, energy, p0 = init_par)

    return best_par

In [ ]:
def fit_and_plot(eos_data):
    """ Function that reads data from a filename and fits a Murnaghan EOS.
    The fitted parameters and a plot are returned. """
    # unpack eos_data
    volume, energy = map(np.array, zip(*eos_data))
    # fit data to Murnaghan EOS
    best_par = fit_murnaghan(volume, energy)
    # print optimal paramaters
    print "Fit parameters:"
    print " V0     =  {:1.4f} A^3 ".format(best_par[3])
    print " E0     =  {:1.4f} eV  ".format(best_par[0])
    print " B(V0)  =  {:1.4f} eV/A^3".format(best_par[1])
    print " B'(VO) =  {:1.4f} ".format(best_par[2])
    # theoretical lattice constant
    # Re_FACTOR these:
    factor = 1./4.
    lattice_const = (best_par[3]/factor)**(1./3.)
    print "Theoretical lattice constant: {:1.4f} A".format(lattice_const)

    # generate Murnaghan model with fitted parameters
    m_volume = np.linspace(volume.min(), volume.max(), 1000) 
    m_energy = eos_murnaghan(m_volume, *best_par) 
    
    # plot data and model together
    lines = plt.plot(volume, energy, 'ok', m_volume, m_energy, '--r' )
    plt.xlabel(r"Volume [$\rm{A}^3$]")
    plt.ylabel(r"Energy [$\rm{eV}$]")

    #return  best_par, lines

In [ ]:
%matplotlib inline
fit_and_plot(eos_data)